# Running the experiments of the paper

## 1. Run the assimilator double gyre experiments on double gyre files

In [ ]:
# Download the data, if necessary

import wget
from pathlib import Path

files_to_download = ["parts_double_gyre_ref_eps_0.25_A_0.1175_as_2.nc", "parts_double_gyre_ref_eps_0.25_A_0.105_as_2.nc",
"parts_double_gyre_ref_eps_0.25_A_0.11_as_2.nc",
"parts_double_gyre_ref_eps_0.25_A_0.125_as_2.nc",
"parts_double_gyre_ref_eps_0.25_A_0.1_as_2.nc",
"parts_double_gyre_ref_eps_0.35_A_0.1_as_2.nc",
"parts_double_gyre_ref_eps_0.3_A_0.1_as_2.nc",
"parts_double_gyre_ref_eps_0.4_A_0.1_as_2.nc",
"parts_double_gyre_ref_eps_0.5_A_0.1_as_2.nc",
"parts_double_gyre_ref_eps_0_A_0.1_as_2.nc",
"parts_double_gyre_ref_eps_1_A_0.1_as_2.nc",
"parts_double_gyre_standard_as_3.nc"]

Path("data/").mkdir(exist_ok=True)
for file_to_download in files_to_download:
  if not Path(f"data/{file_to_download}").exists():
    print(f"Downloading {file_to_download}...")
    wget.download(f"https://zenodo.org/record/4426130/files/{file_to_download}", f"data/{file_to_download}")

print("All files required for experiment have been downloaded")

In [ ]:
from src.run_assimilator import run_assimilator

# Change the following parameters to reproduce the different experimental results
# Changing A and epsilon will require to have downloaded the corresponding files.
# If you are trying values of A and epsilon that do not have a corresponding dispersion file, you will have to generate it with the double gyre code

mu = 2
A = 0.1
epsilon = 0.25
sigma_rel = 0.01
OBSERVATION_LOCATIONS = [(12, 4), (55, 27)]

if __name__ == "__main__":
    run_assimilator(
        particles_dataset_path="data/parts_double_gyre_standard_as_3.nc",
        observations_type="from_simulation",
        observations_source_path=f"data/parts_double_gyre_ref_eps_{epsilon}_A_{A}_as_2.nc",
        simulation_name=f"mu_{mu}_A_{A}_eps_{epsilon}_sigma_rel_{sigma_rel}_obspoints_{OBSERVATION_LOCATIONS}",
        assimilation_domain_coords=(195, 20, 225, 40),
        assimilation_grid_size=(60, 40),
        size_ensemble=10,
        initial_ensemble_spread=0.05,
        t_start=0,
        t_end=2000,
        observations_error_percent=sigma_rel,
        observation_locations=OBSERVATION_LOCATIONS,
        initial_mass_multiplicator=mu,
    )

## 2. Run a self-assimilation experiment with observation sampled a dispersion model assimilated into another dispersion model

In [ ]:
# First, download the data if it's not present, and prepare it

import xarray as xr
import numpy as np
from datetime import datetime
from dask.diagnostics import ProgressBar
import wget
from pathlib import Path

files_to_download = ["advector_output_rivers_2012.nc", "advector_output_coastal_2012.nc"]

Path("data/").mkdir(exist_ok=True)
for file_to_download in files_to_download:
  if not Path(f"data/{file_to_download}").exists():
    print(f"Downloading {file_to_download}...")
    wget.download(f"https://zenodo.org/record/4740138/files/{file_to_download}", f"data/{file_to_download}")

ds_in = xr.open_dataset("data/advector_output_coastal_2012.nc", chunks="auto")
years_release = ds_in.release_date.dt.year
days_release = ds_in.release_date.dt.dayofyear
arg_release = (years_release - 1993) + days_release / 366

def increasing_sine(x):
    return x + (1 / (2 * np.pi)) * np.sin(2 * np.pi * x + np.pi)

# 1 tonne per particle, as sensible value for a global plastic dispersion model
ds_in["weight"] = 1000 + 1000 * increasing_sine(arg_release)
with ProgressBar():
    ds_in.to_netcdf("data/advector_output_coastal_2012_adapted.nc")

In [ ]:
# Run the Ocean Plastic Assimilator

from src.run_assimilator import run_assimilator
import numpy as np

cell_area = lambda dlon,dlat,lat: dlat * 111 * dlon * 111 * np.cos(lat * np.pi / 180)

GRID_COORDS = (-165, 23, -125, 45)
GRID_RESOLUTION = 0.5

cells_area = np.array([[cell_area(0.5, 0.5, lat_id * GRID_RESOLUTION + GRID_COORDS[1]) for lat_id in range(44)] for lon_id in range(80)])

if __name__ == "__main__":
    run_assimilator(
        particles_dataset_path="data/advector_output_rivers_2012.nc",
        observations_type="from_simulation",
        observations_source_path="data/advector_output_coastal_2012_adapted.nc",
        assimilation_domain_coords=GRID_COORDS,
        assimilation_grid_size=(int(40 / GRID_RESOLUTION), int(22 / GRID_RESOLUTION)),
        size_ensemble=10,
        initial_ensemble_spread=50,
        observations_error_percent=0.01,
        observation_locations=[(25, 12), (50, 24)],
        t_start=0,
        t_end=360,
        initial_mass_multiplicator=1000,
        radius_observation=np.inf,
        simulation_name="self_assimilation_gpgp",
        cells_area=cells_area,
    )
